In [7]:
import pandas as pd
from llm_pipeline.pipeline import GenerateEmbeddingsStep, DataPipeline, kNNFilterStep, LLMCallWithDataFrame, LLMCallStep
import csv


C:\Users\forem\Documents\coding\llm-orchestration-1\llm_pipeline\pipeline.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'record_details'] = df.apply(create_record_details, axis=1)
C:\Users\forem\Documents\coding\llm-orchestration-1\llm_pipeline\pipeline.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'prompt'] = df['record_details'].apply(


,name,modified_size,can_fly,good,danger,description
0,aggressive ape,4,-1,0,1,The aggressive ape is a powerful and imposing ...
1,ancient ape,4,-1,0,1,"The ancient ape looms large, a formidable pres..."
2,awful ape,4,-1,-1,1,"The Awful Ape looms at a medium size, its musc..."
3,basic ape,3,-1,0,1,"The Basic Ape is a formidable creature, standi..."
4,big ape,4,-1,0,1,"In the dense jungles, the Big Ape reigns supre..."
5,celestial ape,4,1,1,1,"The Celestial Ape is a majestic, medium-sized ..."
6,thunderous banshee,3,0,-1,1,"The Thunderous Banshee is a menacing specter, ..."
7,tiny banshee,1,0,-1,-1,"The tiny banshee is a mischievous spirit, bare..."
8,tired banshee,1,0,-1,-1,"The tired banshee is a small, ethereal creatur..."
9,toy banshee,0,0,-1,-1,"The Toy Banshee is a diminutive, eerie figure,..."


In [ ]:

# Load test data from CSV file into a DataFrame
try:
    df = pd.read_csv('./data/superData-little.csv')
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")
    raise

reduced_df = df[['name', 'modified_size', 'can_fly', 'good', 'danger']]

# Generate a description for each record in the DataFrame
gen_description = LLMCallStep(
    prompt_template="""The following record is a creature type for a fantasy game. Please provide a description for the creature.
    The size of the creature is a scale from very small (0), to extremely large (10).\
    The can_fly field indicates whether the creature can fly. A value of 1 indicates that the creature can fly, -1 means it cannot, and 0 is not specified.  Different instances of this type may or many not be able to fly.\
    Similarly, the good field indicates whether the creature is good or evil. A value of 1 indicates that the creature is good, -1 means it is evil, and 0 is not specified.\
    The danger field indicates how dangerous the creature is. A value of 1 indicates that the creature is dangerous, -1 means it is not dangerous, and 0 is not specified.\

    
    Please respond with a short creative description of this creature, limit your response to 100 words.

    {record_details}""",
    output_key="description"
)

# Create the pipeline with all steps.
pipeline = DataPipeline(steps=[gen_description])

# Run the pipeline on the DataFrame.
processed_df = pipeline.run(reduced_df)

processed_df.head(20)

In [8]:
#persist the processed data to a CSV file 
# This will allow us save the generated descriptions for later usecd 
processed_df.to_csv('./data/processed_monsters.csv', index=False)

In [ ]:

# Generate one embedding from title & description.
gen_embed = GenerateEmbeddingsStep(
    output_key="embedding_title_desc",
    fields=["title", "description"]
)

# Create the pipeline with all steps.
pipeline = DataPipeline(steps=[gen_embed])

# Run the pipeline on the DataFrame.
processed_df = pipeline.run(df)
count = len(processed_df)

print(f"Processed record count: {count}")
processed_df.to_pickle('./data/monsters_with_embeddings.pkl')


In [4]:
# Reload the DataFrame.
reloaded_df = pd.read_pickle('./data/monsters_with_embeddings.pkl')

# Use a kNN filter on one of the embeddings:
knn_filter = kNNFilterStep(
    query="input form",
    k=3,
    embedding_column="embedding_title_desc"
)

# Create the pipeline with all steps.
pipeline = DataPipeline(steps=[knn_filter])

# Run the pipeline on the DataFrame.
processed_df = pipeline.run(reloaded_df)

print(processed_df[['id', 'title']].to_string())

llm_eval = LLMCallWithDataFrame(
    prompt_template="""What UI updates are being preformed?  Ignore records that not UI updates'.

    {record_details}""",
    fields=["title", "description", "acceptance_criteria"],
)
print(llm_eval.call_llm(processed_df))

        id                                          title
1  B-29990  Change Order Date field name to Shipping Date
2  B-56444      Add support for new data type to back end
0  B-12223                     Query Service improvements
- **Change Order Date field name to Shipping Date**
  - Description: On the Input form, the Order Date label is not correct; this should be shipping date.
  - Acceptance Criteria: Label is updated.
